In [ ]:
%load_ext autoreload

In [ ]:
%matplotlib inline
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

import pandas as pd
import seaborn as sns
import numpy as np

import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split, KFold
from sklearn.grid_search import GridSearchCV
from sklearn.decomposition import NMF

import transform_utils as tu
from kaggle_helper import KaggleHelper

In [ ]:
kh = KaggleHelper("submissions - matrix factorization.db", "predictions")

In [ ]:
%aimport transform_utils
%aimport kaggle_helper

In [ ]:
%aimport

In [ ]:
raw = pd.read_csv("data/train.csv")
print("Unique UPC:", len(raw.Upc.unique()))
print("Unique DepartmentDescription:", len(raw.DepartmentDescription.unique()))
print("Unique FinelineNumber:", len(raw.FinelineNumber.unique()))
raw.head()

In [ ]:
raw = pd.read_csv("data/train.csv")
raw['Returns'] = raw.ScanCount.map(lambda x: abs(x) if x < 0 else 0)
raw.ScanCount = raw.ScanCount.map(lambda x: 0 if x < 0 else x)

In [ ]:
raw.head()

In [ ]:
start = datetime.datetime.now()
%autoreload
group_cols = ['DepartmentDescription', 'Weekday']
keep_cols = ['VisitNumber', 'ScanCount', 'Returns']
df = tu.preprocess_walmart(raw, group_cols, keep_cols, True)

print("Total Time:", (datetime.datetime.now() - start).total_seconds())

In [ ]:
df.columns

In [ ]:
X = df.drop("TripType", axis=1).drop("VisitNumber", axis=1)
sorted_columns = sorted(X.columns)
X = X[sorted_columns]
y = df.TripType

In [ ]:
mf = NMF()
mfX = mf.fit_transform(X.values)

In [ ]:
print(mfX.shape)
print(y.shape)

In [ ]:
XTrain, XVal, yTrain, yVal = train_test_split(mfx, y.values, test_size=0.2, random_state=200)

http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html#sklearn.grid_search.GridSearchCV

In [ ]:
start = datetime.datetime.now()
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(XTrain, yTrain)

score = clf.score(XVal, yVal)
print(score)

print("Total Time:", (datetime.datetime.now() - start).total_seconds())

In [ ]:
kh.save_validation_score(clf, score, len(XTrain))

# Only Kaggle Exists Below Here

In [ ]:
kaggleTestRaw = pd.read_csv("data/test.csv")
kaggleTest = tu.preprocess_walmart(kaggleTestRaw,
                                   group_cols, 
                                   keep_cols)
missing_columns = set(sorted_columns).difference(set(kaggleTest.columns))
print("missing columns")
print("-"*15)
for col in missing_columns:
    print(col)
    kaggleTest[col] = 0
kaggleTest = kaggleTest[sorted_columns]
assert list(X.columns) == list(kaggleTest.columns)

In [ ]:
%autoreload
test_predictions = clf.predict(kaggleTest.values)
output = tu.convert_predictions(test_predictions, 
                               output_index=pd.Series(kaggleTest.index), 
                               actual_trip_types=df.TripType.unique())

In [ ]:
kh.save_test_predictions(output, clf, len(XTrain))